Dans ce fichier, nous allons essayer de classifier les lésions en utilisant un réseau de neurones convolutif suivi d'une SVM.
#### Importations

In [118]:
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.externals import joblib
from sklearn.metrics import matthews_corrcoef 
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from time import time
from sklearn.ensemble import RandomForestClassifier as RDF
import cv2
from skimage.io import imread
from skimage.io import imsave

In [84]:
### On change la taille des Images####
t = time()
feat_df = pd.read_csv('features.csv', sep=';')

# On parcours toutes les images du dossier
for x in feat_df['ImageId'].values:
    filename = 'im/{}.jpg'.format(x)
    filename_S = 'im/{}_Segmentation.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S, as_grey = True)
    #on applique le masque
    image = cv2.bitwise_and(image, image, mask = image_S)
    # on cahnge la taille        
    seg_image = cv2.resize(image,(149,149))
    imsave('im_resized2/{}_Segmentation_resized.jpg'.format(x), seg_image)
    
print('Temps de calcul : ', time()-t)

Temps de calcul :  115.13847923278809


Fonctions tirées de https://code.oursky.com/tensorflow-svm-image-classifications-engine/

La première initialise le réseau et la deuxième calcule les features

In [5]:
def create_graph(model_path):
    """
    create_graph loads the inception model to memory, should be called before
    calling extract_features.
 
    model_path: path to inception model in protobuf form.
    """
    with gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
 
 
def extract_features(image_paths, verbose=False):
    """
    extract_features computed the inception bottleneck feature for a list of images
 
    image_paths: array of image path
    return: 2-d array in the shape of (len(image_paths), 2048)
    """
    feature_dimension = 2048
    features = np.empty((len(image_paths), feature_dimension))
 
    with tf.Session() as sess:
        flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')
 
        for i, image_path in enumerate(image_paths):
            if verbose:
                print('Processing %s...' % (image_path))
 
            if not gfile.Exists(image_path):
                tf.logging.fatal('File does not exist %s', image)
            
            #print(image_path)
            if (i%100 == 0):
                print("Processing image n° :", i)
            image_data = gfile.FastGFile(image_path, 'rb').read()
            feature = sess.run(flattened_tensor, {
                'DecodeJpeg/contents:0': image_data
            })
            features[i, :] = np.squeeze(feature)
 
    return features 


In [6]:
df = pd.read_csv('all.csv', sep=',')
# all.csv contient les les identifiants de toutes les images. Les 300 premiers sont ceux du test set et 
# les 600 autres ceux du training set
X_df = df['ImageId'].values

# le nom de toutes les images qui vont être entrées dans le réseau
images_dir = 'im_resized2/' + X_df + '_Segmentation_resized.jpg'
create_graph('tensorflow_inception_graph.pb')
features = extract_features(images_dir)

print(features.shape)

Processing image n° : 0
Processing image n° : 100
Processing image n° : 200
Processing image n° : 300
Processing image n° : 400
Processing image n° : 500
Processing image n° : 600
Processing image n° : 700
Processing image n° : 800
(900, 2048)


In [146]:
df_train = pd.read_csv('train.csv', sep=',')
Y_train = df_train['Malignant'].values
X_train = features[300:, :]
X_test = features[:300, :]
# on va centrer et réduire les données
scale = StandardScaler()

In [147]:
#on fait un pipeline pour trouver les meilleurs paramètres avec la pca
pipe = Pipeline(steps=[('pca', PCA()), ('svc', SVC(kernel='rbf'))])

estimator = GridSearchCV(pipe, dict(pca__n_components=[100, 500, 1000,2048], svc__C= [10, 100],
                                    svc__gamma = [ 1e-2,1e-3, 1e-4]), scoring = make_scorer(matthews_corrcoef))
estimator.fit(scale.fit_transform(X_train), Y_train)
print(estimator.best_params_)
print("Score : ", estimator.best_score_)

/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


{'pca__n_components': 100, 'svc__C': 10, 'svc__gamma': 0.0001}
Score :  0.338313212061


On enregistre la prediction

In [148]:
df_test = pd.read_csv('test.csv', sep=',')
df_test['Malignant'] = estimator.predict(scale.transform(X_test))
df_test.to_csv('test.csv', index=None, sep=',', mode='w') 

Le score obtenu est de 0.37007